In [ ]:
import os
os.chdir("e:\GITHUB\Crime investigate\data")#change directory 


In [11]:
%pwd #working directory

'e:\\'

In [3]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


e:\GITHUB\Crime investigate\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_pdf_files(data):
  loader = DirectoryLoader(
    data,                     #path
    glob="*.pdf",             #only the pdfs(all pdfs)
    loader_cls=PyPDFLoader    #to load pdf documents langchain help
  )
  documents = loader.load()
  return documents

In [ ]:
extracted_data=load_pdf_files(r"e:\GITHUB\Crime investigate\data")

In [21]:
extracted_data


[Document(metadata={'producer': 'Acrobat Distiller 3.01 for Power Macintosh', 'creator': 'PageMaker 6.5', 'creationdate': 'D:20010423131854', 'author': 'National Institute of Justice', 'title': 'Crime Scene Investigation: A Guide for Law Enforcement', 'moddate': '2001-06-21T11:14:53-04:00', 'subject': 'Criminal investigation, evidence', 'keywords': 'Criminal investigation, evidence', 'source': 'data\\crime_investigate.pdf', 'total_pages': 58, 'page': 0, 'page_label': '1'}, page_content='U.S. Department of Justice\nOffice of Justice Programs\nNational Institute of Justice\nresearch report\nA Guide for Law EnforcementA Guide for Law Enforcement\nCrime Scene\nInvestigation\nCrime Scene\nInvestigation'),
 Document(metadata={'producer': 'Acrobat Distiller 3.01 for Power Macintosh', 'creator': 'PageMaker 6.5', 'creationdate': 'D:20010423131854', 'author': 'National Institute of Justice', 'title': 'Crime Scene Investigation: A Guide for Law Enforcement', 'moddate': '2001-06-21T11:14:53-04:00'

In [22]:
#filter to get only wanted (ex remove meta data)
from typing import List
from langchain.schema import Document

def filter_to_minimal_doc(docs: List[Document]) -> List[Document]:
    
    """Filter document objs to return a newlist of doc objects to only 
    include 'source' in metadata and page content."""

    minimal_docs: List[Document] = []
    for doc in docs:
        src= doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [23]:
minimal_doc=filter_to_minimal_doc(extracted_data)

In [24]:
minimal_doc

[Document(metadata={'source': 'data\\crime_investigate.pdf'}, page_content='U.S. Department of Justice\nOffice of Justice Programs\nNational Institute of Justice\nresearch report\nA Guide for Law EnforcementA Guide for Law Enforcement\nCrime Scene\nInvestigation\nCrime Scene\nInvestigation'),
 Document(metadata={'source': 'data\\crime_investigate.pdf'}, page_content='U.S. Department of Justice\nOffice of Justice Programs\n810 Seventh Street N.W.\nWashington, DC 20531\nJanet Reno\nAttorney General\nDaniel Marcus\nActing Associate Attorney General\nLaurie Robinson\nAssistant Attorney General\nNoël Brennan\nDeputy Assistant Attorney General\nJeremy Travis\nDirector, National Institute of Justice\nDepartment of Justice Response Center\n800–421–6770\nOffice of Justice Programs National Institute of Justice\nWorld Wide Web Site World Wide Web Site\nhttp://www.ojp.usdoj.gov http://www.ojp.usdoj.gov/nij\nCover photograph of man on the ground by Corbis Images. Other cover photographs\ncopyright

In [25]:
#split documents into small chunks
def text_split(minimal_doc):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=20,      #to understand context with chunks
    )
  texts_chunk=text_splitter.split_documents(minimal_doc)
  return texts_chunk

In [28]:
text_chunk=text_split(minimal_doc)
print(f"Number of chunks {len(text_chunk)}")

Number of chunks 192


In [27]:
print(text_chunk)

[Document(metadata={'source': 'data\\crime_investigate.pdf'}, page_content='U.S. Department of Justice\nOffice of Justice Programs\nNational Institute of Justice\nresearch report\nA Guide for Law EnforcementA Guide for Law Enforcement\nCrime Scene\nInvestigation\nCrime Scene\nInvestigation'), Document(metadata={'source': 'data\\crime_investigate.pdf'}, page_content='U.S. Department of Justice\nOffice of Justice Programs\n810 Seventh Street N.W.\nWashington, DC 20531\nJanet Reno\nAttorney General\nDaniel Marcus\nActing Associate Attorney General\nLaurie Robinson\nAssistant Attorney General\nNoël Brennan\nDeputy Assistant Attorney General\nJeremy Travis\nDirector, National Institute of Justice\nDepartment of Justice Response Center\n800–421–6770\nOffice of Justice Programs National Institute of Justice\nWorld Wide Web Site World Wide Web Site'), Document(metadata={'source': 'data\\crime_investigate.pdf'}, page_content='http://www.ojp.usdoj.gov http://www.ojp.usdoj.gov/nij\nCover photogra

In [31]:
#Embeddding - using huggingface model
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
        )
    return embeddings

embedding=download_embeddings()

In [32]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [33]:
vector=embedding.embed_query("Hello World")

In [34]:
vector

[-0.034477293491363525,
 0.031023185700178146,
 0.006734919268637896,
 0.026108955964446068,
 -0.03936200588941574,
 -0.16030244529247284,
 0.06692398339509964,
 -0.00644145580008626,
 -0.047450482845306396,
 0.014758873730897903,
 0.07087531685829163,
 0.05552761256694794,
 0.019193336367607117,
 -0.026251327246427536,
 -0.010109526105225086,
 -0.026940451934933662,
 0.02230745740234852,
 -0.02222668007016182,
 -0.14969263970851898,
 -0.017492998391389847,
 0.007676251698285341,
 0.05435226485133171,
 0.003254401497542858,
 0.031725890934467316,
 -0.08462139964103699,
 -0.029405971989035606,
 0.051595598459243774,
 0.04812406003475189,
 -0.003314854810014367,
 -0.05827920511364937,
 0.04196922481060028,
 0.022210687398910522,
 0.1281888335943222,
 -0.02233893983066082,
 -0.011656275019049644,
 0.06292839348316193,
 -0.032876357436180115,
 -0.0912260189652443,
 -0.03117534890770912,
 0.05269956961274147,
 0.04703487083315849,
 -0.08420306444168091,
 -0.030056191608309746,
 -0.020744830

In [35]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [36]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY

In [39]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [40]:
pc

In [ ]:
from pinecone import ServerlessSpec 

index_name="crime-investigate"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,                   #dimension of the embedding vector
        metric="cosine",                 #similarity measure
        spec=ServerlessSpec(cloud="aws",region="us-east-1") #serverless configuration
    )

TypeError: Pinecone.create_index() got an unexpected keyword argument 'serverless'